## Topics that we'll be covering are:
1. Essential Theory of RNN's
2. RNN's in Practice
3. Essential theory of LSTM's and GRU's
4. LSTM and GRU's in Practice

## GRU(Gated Recurrent Unit)

#### Resources for theory:
1. Brendon Rohrer's Youtube video
2. Andrej Karpathy's Blog
3. Chris Olah's Github Blog

In [1]:
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, SpatialDropout1D
from keras.layers import GRU
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

/home/mohammedsunasra/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mohammedsunasra/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
output_dir = "../model_files/gru/"

epochs = 4
batch_size = 128
n_dim = 64
n_unique_words = 10000

max_review_length = 100

pad_type = trunc_type = 'pre'
drop_emb = 0.2

n_grus = 256
drop_gru = 0.2

#### Load Data

In [5]:
(X_train, y_train), (X_val, y_val) = imdb.load_data(num_words=n_unique_words)

#### Preprocess Data

In [6]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_val = pad_sequences(X_val, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design Neural Net Architecture

In [9]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim,input_length=max_review_length))
model.add(SpatialDropout1D(drop_emb))
model.add(GRU(n_grus, dropout=drop_gru))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 100, 64)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               246528    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 886,785
Trainable params: 886,785
Non-trainable params: 0
_________________________________________________________________


#### Configure Model


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Train!!

In [12]:
model_checkpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [13]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val), \
         callbacks=[model_checkpoint])

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 375s 15ms/step - loss: 0.5387 - acc: 0.7057 - val_loss: 0.4274 - val_acc: 0.8218
Epoch 2/4
25000/25000 [==============================] - 296s 12ms/step - loss: 0.3138 - acc: 0.8696 - val_loss: 0.3716 - val_acc: 0.8378
Epoch 3/4
25000/25000 [==============================] - 295s 12ms/step - loss: 0.2558 - acc: 0.8979 - val_loss: 0.3503 - val_acc: 0.8478
Epoch 4/4
25000/25000 [==============================] - 145s 6ms/step - loss: 0.2012 - acc: 0.9240 - val_loss: 0.4083 - val_acc: 0.8421
